#### 결측치 대체
mean: 평균  
meadian: 중앙값  
mode: 최빈값  

---
1. `MissingValueHandler` 클래스:
   - 데이터 로드, 결측치 정보 조회
   - 다양한 방법(평균값, 중앙값, 최빈값)으로 결측치 처리
   - 처리 히스토리 저장 및 취소(undo) 기능
   - 결과 저장 및 JSON 형식 변환

2. `api_handle_missing_values` 함수:
   - 웹 서비스 API 응답 형식 시뮬레이션
   - 상태, 메시지, 변경 데이터 등 반환

사용자 흐름:
1. 사용자가 타이타닉 데이터를 업로드
2. Age와 같은 결측치를 포함한 칼럼 선택
3. 대체 방법(평균값, 중앙값, 최빈값) 선택
4. 백엔드 API 요청 → 결측치 처리 → 변경된 행, 원래 행, 인덱스 반환
5. '데이터 미리보기' 섹션에 변경된 행 표시
6. 사용자는 작업 취소 가능 (취소 시 이전 화면으로 복귀)

In [8]:
import pandas as pd
import numpy as np
import json
import os
from datetime import datetime

In [9]:
class MissingValueHandler:
    def __init__(self, data_path=None, df=None):
        """
        결측치 처리 클래스 초기화
        
        Parameters:
        -----------
        data_path : str, optional
            데이터 파일 경로
        df : pandas.DataFrame, optional
            직접 데이터프레임 전달 시 사용
        """
        if df is not None:
            self.df = df.copy()
        elif data_path is not None:
            self.df = pd.read_csv(data_path)
        else:
            raise ValueError("데이터 파일 경로 또는 데이터프레임을 제공해야 합니다.")
            
        # 데이터 처리 히스토리 저장
        self.history = []
        # 원본 데이터 백업
        self.original_df = self.df.copy()
        
        # 결과 저장 디렉토리 생성
        self.result_dir = 'preprocessing_results'
        os.makedirs(self.result_dir, exist_ok=True)
    
    def get_missing_info(self):
        """결측치 정보 조회"""
        missing_values = self.df.isnull().sum()
        missing_ratio = (missing_values / len(self.df)) * 100
        missing_data = pd.concat([missing_values, missing_ratio], axis=1)
        missing_data.columns = ['결측치 수', '결측치 비율(%)']
        
        # 결측치가 있는 컬럼만 필터링
        missing_data = missing_data[missing_data['결측치 수'] > 0]
        
        # 각 컬럼별 결측치 정보를 담은 딕셔너리 생성
        result = {}
        for idx, row in missing_data.iterrows():
            result[idx] = {
                '결측치 수': int(row['결측치 수']),
                '결측치 비율(%)': float(row['결측치 비율(%)'])
            }
        
        return result
    
    def handle_missing_values(self, column, method='mean', save_history=True):
        """
        선택한 컬럼의 결측치 처리
        
        Parameters:
        -----------
        column : str
            처리할 컬럼 이름
        method : str, default='mean'
            대체 방법 ('mean', 'median', 'mode')
        save_history : bool, default=True
            처리 히스토리 저장 여부
            
        Returns:
        --------
        dict
            처리 결과 정보
        """
        if column not in self.df.columns:
            return {"error": f"컬럼 '{column}'이 데이터에 존재하지 않습니다."}
            
        # 결측치가 있는 행 인덱스 저장
        missing_indices = self.df[self.df[column].isnull()].index.tolist()
        
        if not missing_indices:
            return {"error": f"컬럼 '{column}'에 결측치가 없습니다."}
            
        # 원래 데이터의 복사본 저장
        original_rows = self.df.loc[missing_indices].copy()
        
        # 데이터 타입 확인
        is_numeric = pd.api.types.is_numeric_dtype(self.df[column])
        
        # 결측치 대체 방법에 따라 처리
        if method == 'mean':
            if not is_numeric:
                return {"error": f"'{column}' 컬럼은 숫자형이 아니므로 평균값 대체가 불가능합니다."}
            fill_value = self.df[column].mean()
            description = f"{column} 컬럼의 결측치를 평균값({fill_value:.2f})으로 대체"
        elif method == 'median':
            if not is_numeric:
                return {"error": f"'{column}' 컬럼은 숫자형이 아니므로 중앙값 대체가 불가능합니다."}
            fill_value = self.df[column].median()
            description = f"{column} 컬럼의 결측치를 중앙값({fill_value:.2f})으로 대체"
        elif method == 'mode':
            fill_value = self.df[column].mode()[0]
            description = f"{column} 컬럼의 결측치를 최빈값({fill_value})으로 대체"
        else:
            return {"error": "유효하지 않은 대체 방법입니다. 'mean', 'median', 'mode' 중 하나를 사용하세요."}
            
        # 결측치 대체
        self.df[column].fillna(fill_value, inplace=True)
        
        # 변경된 행
        changed_rows = self.df.loc[missing_indices].copy()
        
        # 처리 결과
        result = {
            "column": column,
            "method": method,
            "fill_value": str(fill_value) if not isinstance(fill_value, (int, float)) else float(fill_value) if not pd.isna(fill_value) else None,
            "changed_indices": missing_indices,
            "original_rows": original_rows.to_dict('records'),
            "changed_rows": changed_rows.to_dict('records'),
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        
        # 히스토리 저장
        if save_history:
            self.history.append(result)
            # 결과 파일 저장
            self._save_result(result)
            
        return result
    
    def undo_last_operation(self):
        """가장 최근 처리 작업 취소"""
        if not self.history:
            return {"error": "취소할 작업이 없습니다."}
            
        # 가장 최근 작업 가져오기
        last_op = self.history.pop()
        column = last_op["column"]
        indices = last_op["changed_indices"]
        
        # 원래 값으로 복원
        for idx, row in enumerate(last_op["original_rows"]):
            self.df.loc[indices[idx], column] = row[column]
            
        return {
            "message": f"{column} 컬럼의 결측치 처리가 취소되었습니다.",
            "restored_rows": last_op["original_rows"]
        }
    
    def get_current_data(self):
        """현재 데이터프레임 반환"""
        return self.df.copy()
    
    def get_history(self):
        """처리 히스토리 반환"""
        return self.history
    
    def reset_to_original(self):
        """원본 데이터로 초기화"""
        self.df = self.original_df.copy()
        self.history = []
        return {"message": "데이터가 원본 상태로 초기화되었습니다."}
    
    def _save_result(self, result):
        """처리 결과 저장"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{self.result_dir}/missing_value_{result['column']}_{result['method']}_{timestamp}.json"
        
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(result, f, ensure_ascii=False, indent=2)
            
        return filename
    
    def save_dataframe(self, filename=None):
        """현재 데이터프레임 저장"""
        if filename is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"{self.result_dir}/processed_data_{timestamp}.csv"
            
        self.df.to_csv(filename, index=False)
        return {"message": f"데이터가 '{filename}'에 저장되었습니다."}
    
    def export_as_json(self):
        """현재 데이터프레임을 JSON 형식으로 변환"""
        return {
            "data": self.df.to_dict('records'),
            "columns": list(self.df.columns),
            "shape": self.df.shape,
            "missing_info": self.get_missing_info()
        }

#### API 응답을 시뮬레이션하는 함수

In [10]:
def api_handle_missing_values(df, column, method):
    """
    결측치 처리 API 응답 시뮬레이션
    
    Parameters:
    -----------
    df : pandas.DataFrame
        데이터프레임
    column : str
        처리할 컬럼 이름
    method : str
        대체 방법 ('mean', 'median', 'mode')
        
    Returns:
    --------
    dict
        API 응답 형식의 결과
    """
    handler = MissingValueHandler(df=df)
    result = handler.handle_missing_values(column, method)
    
    if "error" in result:
        return {
            "status": "error",
            "message": result["error"],
            "data": None
        }
    
    return {
        "status": "success",
        "message": f"{column} 컬럼의 결측치를 {method} 방식으로 처리했습니다.",
        "data": {
            "column": column,
            "method": method,
            "fill_value": result["fill_value"],
            "changed_indices": result["changed_indices"],
            "original_rows": result["original_rows"],
            "changed_rows": result["changed_rows"],
            "processed_df": handler.get_current_data().to_dict('records')
        }
    }

#### 테스트 및 사용 예제

In [11]:
if __name__ == "__main__":
    # 데이터 로드
    titanic_url = 'https://bit.ly/fc-ml-titanic'
    df = pd.read_csv(titanic_url)
    
    print("===== 타이타닉 데이터 결측치 처리 예제 =====")
    
    # 결측치 처리 인스턴스 생성
    handler = MissingValueHandler(df=df)
    
    # 결측치 정보 확인
    missing_info = handler.get_missing_info()
    print("\n1. 결측치 정보:")
    for col, info in missing_info.items():
        print(f"  - {col}: {info['결측치 수']}개 ({info['결측치 비율(%)']:.2f}%)")
    
    # 예제 1: Age 컬럼의 결측치를 평균값으로 대체
    print("\n2. Age 컬럼 결측치 평균값으로 대체:")
    result1 = handler.handle_missing_values('Age', 'mean')
    print(f"  - 대체값: {result1['fill_value']:.2f}")
    print(f"  - 변경된 행 수: {len(result1['changed_indices'])}")
    
    # 변경된 첫 번째 행 예시 출력
    if result1['changed_indices']:
        idx = result1['changed_indices'][0]
        print(f"\n  변경 전: {result1['original_rows'][0]['Age']}")
        print(f"  변경 후: {result1['changed_rows'][0]['Age']}")
    
    # 히스토리 확인
    print("\n3. 처리 히스토리:")
    history = handler.get_history()
    for i, h in enumerate(history):
        print(f"  작업 {i+1}: {h['column']} 컬럼 {h['method']} 방식 처리 ({h['timestamp']})")
    
    # 예제 2: Embarked 컬럼의 결측치를 최빈값으로 대체
    print("\n4. Embarked 컬럼 결측치 최빈값으로 대체:")
    result2 = handler.handle_missing_values('Embarked', 'mode')
    if "error" in result2:
        print(f"  - 오류: {result2['error']}")
    else:
        print(f"  - 대체값: {result2['fill_value']}")
        print(f"  - 변경된 행 수: {len(result2['changed_indices'])}")
    
    # API 응답 시뮬레이션
    print("\n5. API 응답 형식 예제:")
    api_response = api_handle_missing_values(df, 'Cabin', 'mode')
    print(f"  상태: {api_response['status']}")
    print(f"  메시지: {api_response['message']}")
    print(f"  변경된 행 수: {len(api_response['data']['changed_indices'])}")
    
    # 마지막 작업 취소
    print("\n6. 마지막 작업 취소:")
    undo_result = handler.undo_last_operation()
    print(f"  메시지: {undo_result['message']}")
    
    # 결과 저장
    print("\n7. 처리된 데이터 저장:")
    save_result = handler.save_dataframe()
    print(f"  {save_result['message']}")
    
    print("\n===== 테스트 완료 =====")

===== 타이타닉 데이터 결측치 처리 예제 =====

1. 결측치 정보:
  - Age: 177개 (19.87%)
  - Cabin: 687개 (77.10%)
  - Embarked: 2개 (0.22%)

2. Age 컬럼 결측치 평균값으로 대체:
  - 대체값: 29.70
  - 변경된 행 수: 177

  변경 전: nan
  변경 후: 29.69911764705882

3. 처리 히스토리:
  작업 1: Age 컬럼 mean 방식 처리 (2025-04-26 22:57:58)

4. Embarked 컬럼 결측치 최빈값으로 대체:
  - 대체값: S
  - 변경된 행 수: 2

5. API 응답 형식 예제:
  상태: success
  메시지: Cabin 컬럼의 결측치를 mode 방식으로 처리했습니다.
  변경된 행 수: 687

6. 마지막 작업 취소:
  메시지: Embarked 컬럼의 결측치 처리가 취소되었습니다.

7. 처리된 데이터 저장:
  데이터가 'preprocessing_results/processed_data_20250426_225758.csv'에 저장되었습니다.

===== 테스트 완료 =====


C:\Users\SSAFY\AppData\Local\Temp\ipykernel_13888\2998187165.py:100: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  self.df[column].fillna(fill_value, inplace=True)
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_13888\2998187165.py:100: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For